In [ ]:
!pip install keras==2.0.0
!pip install tensorflow==1.15

     |████████████████████████████████| 191 kB 5.4 MB/s 
     |████████████████████████████████| 462 kB 46.4 MB/s 
     |████████████████████████████████| 495.0 MB 30 kB/s 
     |████████████████████████████████| 489.6 MB 28 kB/s 
     |████████████████████████████████| 463.8 MB 34 kB/s 
     |████████████████████████████████| 14.8 MB 33.7 MB/s 
     |████████████████████████████████| 132 kB 45.3 MB/s 
     |████████████████████████████████| 458.3 MB 13 kB/s 
     |████████████████████████████████| 458.3 MB 14 kB/s 
     |████████████████████████████████| 458.3 MB 12 kB/s 
     |████████████████████████████████| 460.3 MB 9.7 kB/s 
     |████████████████████████████████| 1.2 MB 34.8 MB/s 
     |████████████████████████████████| 4.0 MB 36.9 MB/s 
     |████████████████████████████████| 462 kB 44.0 MB/s 
  Created wheel for keras: filename=Keras-2.0.0-py3-none-any.whl size=227982 sha256=04884b3fd301f1bc516c89a00140f0c0b9b60d85f7f128e47597dab222423dbb
  Stored in directory: /root/.cache/pi

     |████████████████████████████████| 412.3 MB 26 kB/s 
     |████████████████████████████████| 3.8 MB 28.3 MB/s 
     |████████████████████████████████| 503 kB 44.2 MB/s 
     |████████████████████████████████| 50 kB 4.6 MB/s 
  Created wheel for gast: filename=gast-0.2.2-py3-none-any.whl size=7554 sha256=534a5ab2252383944722adaf4d2d2067943a0548407be08580aea0564bd37eae
  Stored in directory: /root/.cache/pip/wheels/21/7f/02/420f32a803f7d0967b48dd823da3f558c5166991bfd204eef3
Successfully built gast
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.5.0
    Uninstalling tensorflow-estimator-2.5.0:
      Successfully uninstalled tensorflow-estimator-2.5.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.8.0
    Uninstalling tensorboard-2.8.0:
      Successfully uninstalled tensorboard-2.8.0
  Attempting uninstall: gast
    Found existing installation: gast 0.4.0
    Uninstalling gast-0.4.0:
      Succes

In [ ]:
import numpy as np
import keras
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Input, Lambda
from keras.layers.merge import Maximum
from keras.layers.convolutional import Conv2D, MaxPooling2D
from tensorflow.keras.layers import BatchNormalization
from itertools import combinations
from collections import defaultdict
import tensorflow as tf
from keras.layers.merge import _Merge
from tensorflow.keras.utils import to_categorical

Using TensorFlow backend.


In [ ]:
import numpy as np
import keras
from keras.models import Model
from keras.layers import Dense, Flatten, Input, Lambda
from keras.layers.merge import Maximum
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from itertools import combinations
from collections import defaultdict
import tensorflow as tf 
from keras.layers.merge import _Merge


In [ ]:

class MatchCosine(_Merge):
    """
        Matching network with cosine similarity metric
    """
    def __init__(self,nway=5,**kwargs):
        super(MatchCosine,self).__init__(**kwargs)
        self.eps = 1e-10
        self.nway = nway

    def build(self, input_shape):
        if not isinstance(input_shape, list) or len(input_shape) != self.nway+2:
            raise ValueError('A ModelCosine layer should be called on a list of inputs of length %d'%(self.nway+2))

    def call(self,inputs):
        """
        inputs in as array which contains the support set the embeddings, 
        the target embedding as the second last value in the array, and true class of target embedding as the last value in the array
        """ 
        similarities = []

        targetembedding = inputs[-2] # embedding of the query image
        numsupportset = len(inputs)-2
        for ii in range(numsupportset):
            supportembedding = inputs[ii] # embedding for i^{th} member in the support set

            sum_support = tf.reduce_sum(tf.square(supportembedding), 1, keep_dims=True)
            supportmagnitude = tf.rsqrt(tf.clip_by_value(sum_support, self.eps, float("inf"))) #reciprocal of the magnitude of the member of the support 

            sum_query = tf.reduce_sum(tf.square(targetembedding), 1, keep_dims=True)
            querymagnitude = tf.rsqrt(tf.clip_by_value(sum_query, self.eps, float("inf"))) #reciprocal of the magnitude of the query image

            dot_product = tf.matmul(tf.expand_dims(targetembedding,1),tf.expand_dims(supportembedding,2))
            dot_product = tf.squeeze(dot_product,[1])

            cosine_similarity = dot_product*supportmagnitude*querymagnitude
            similarities.append(cosine_similarity)

        similarities = tf.concat(axis=1,values=similarities)
        softmax_similarities = tf.nn.softmax(similarities)
        preds = tf.squeeze(tf.matmul(tf.expand_dims(softmax_similarities,1),inputs[-1]))
        
        preds.set_shape((inputs[0].shape[0],self.nway))

        return preds

    def compute_output_shape(self,input_shape):
        input_shapes = input_shape
        return (input_shapes[0][0],self.nway)

# Bonus: Matching network with Euclidean metrtic
class MatchEuclidean(_Merge):
    """
        Matching network with Euclidean metric
    """
    def __init__(self,nway=5,**kwargs):
        super(MatchEuclidean,self).__init__(**kwargs)
        self.eps = 1e-10
        self.nway = nway

    def build(self, input_shape):
        if not isinstance(input_shape, list) or len(input_shape) != self.nway+2:
            raise ValueError('A ModelEuclidean layer should be called on a list of inputs of length %d'%(self.nway+2))

    def call(self,inputs):
        """
        inputs in as array which contains the support set the embeddings, the target embedding as the second last value in the array, and true class of target embedding as the last value in the array
        """ 
        similarities = []

        targetembedding = inputs[-2]
        numsupportset = len(inputs)-2
        for ii in range(numsupportset):
            supportembedding = inputs[ii]
            dd = tf.negative(tf.sqrt(tf.reduce_sum(tf.square(supportembedding-targetembedding),1,keep_dims=True)))

            similarities.append(dd)

        similarities = tf.concat(axis=1,values=similarities)
        softmax_similarities = tf.nn.softmax(similarities)
        preds = tf.squeeze(tf.matmul(tf.expand_dims(softmax_similarities,1),inputs[-1]))
        
        preds.set_shape((inputs[0].shape[0],self.nway))

        return preds

    def compute_output_shape(self,input_shape):
        input_shapes = input_shape
        return (input_shapes[0][0],self.nway)

# Siamese network like interaction
class Siamify(_Merge):
    def _merge_function(self,inputs):
        return tf.negative(tf.abs(inputs[0]-inputs[1]))

In [ ]:
np.random.seed(2191)  # for reproducibility

class OmniglotNShotDataset():
    def __init__(self,batch_size,classes_per_set=5,samples_per_class=1,trainsize=32000,valsize=10000):

        """
        Constructs an N-Shot omniglot Dataset
        :param batch_size: Experiment batch_size
        :param classes_per_set: Integer indicating the number of classes per set
        :param samples_per_class: Integer indicating samples per class
        e.g. For a 20-way, 1-shot learning task, use classes_per_set=20 and samples_per_class=1
             For a 5-way, 10-shot learning task, use classes_per_set=5 and samples_per_class=10
        """
        self.x = np.load("/content/drive/MyDrive/Data_colab/data.npy")
        self.x = np.reshape(self.x, [-1, 20, 28, 28, 1])
        shuffle_classes = np.arange(self.x.shape[0])
        np.random.shuffle(shuffle_classes)
        self.x = self.x[shuffle_classes]
        self.x_train, self.x_val  = self.x[:1200], self.x[1200:]
        self.normalization()

        self.batch_size = batch_size
        self.n_classes = self.x.shape[0]
        self.classes_per_set = classes_per_set
        self.samples_per_class = samples_per_class

        self.indexes = {"train": 0, "val": 0}
        self.datasets = {"train": self.x_train, "val": self.x_val}
        self.datasets_cache = {"train": self.packslice(self.datasets["train"],trainsize),
                               "val": self.packslice(self.datasets["val"],valsize)}

    def normalization(self):
        """
        Normalizes our data, to have a mean of 0 and sd of 1
        """
        self.mean = np.mean(self.x_train)
        self.std = np.std(self.x_train)
        self.max = np.max(self.x_train)
        self.min = np.min(self.x_train)
        print("train_shape", self.x_train.shape, "val_shape", self.x_val.shape)
        print("before_normalization", "mean", self.mean, "max", self.max, "min", self.min, "std", self.std)
        self.x_train = (self.x_train - self.mean) / self.std
        self.x_val = (self.x_val - self.mean) / self.std
        self.mean = np.mean(self.x_train)
        self.std = np.std(self.x_train)
        self.max = np.max(self.x_train)
        self.min = np.min(self.x_train)
        print("after_normalization", "mean", self.mean, "max", self.max, "min", self.min, "std", self.std)
        
    def packslice(self, data_pack, numsamples):
        """
        Collects 1000 batches data for N-shot learning
        :param data_pack: Data pack to use (any one of train, val, test)
        :return: A list with [support_set_x, support_set_y, target_x, target_y] ready to be fed to our networks
        """
        n_samples = self.samples_per_class * self.classes_per_set
        support_cacheX = []
        support_cacheY = []
        target_cacheY = []
        
        for iiii in range(numsamples):
            slice_x = np.zeros((n_samples+1,28,28,1))
            slice_y = np.zeros((n_samples,))
            
            ind = 0
            pinds = np.random.permutation(n_samples)
            classes = np.random.choice(data_pack.shape[0],self.classes_per_set,False) # chosen classes
            
            x_hat_class = np.random.randint(self.classes_per_set) # target class
            
            for j, cur_class in enumerate(classes):  # each class
                example_inds = np.random.choice(data_pack.shape[1],self.samples_per_class,False)
                
                for eind in example_inds:
                    slice_x[pinds[ind],:,:,:] = data_pack[cur_class][eind]
                    slice_y[pinds[ind]] = j
                    ind += 1
                
                if j == x_hat_class:
                    slice_x[n_samples,:,:,:] = data_pack[cur_class][np.random.choice(data_pack.shape[1])]
                    target_y = j

            support_cacheX.append(slice_x)
            support_cacheY.append(keras.utils.to_categorical(slice_y,self.classes_per_set))
            target_cacheY.append(keras.utils.to_categorical(target_y,self.classes_per_set)[0])
            
        return np.array(support_cacheX), np.array(support_cacheY), np.array(target_cacheY)

In [ ]:

bsize = 32 # batch size
classes_per_set = 5 # classes per set or 5-way
samples_per_class = 1 # samples per class 1-shot

data = OmniglotNShotDataset(batch_size=bsize,classes_per_set=classes_per_set,samples_per_class=samples_per_class,trainsize=64000,valsize=20000)

# Image embedding using Deep Convolutional Network
conv1 = Conv2D(64,(3,3),padding='same',activation='relu')
bnorm1 = BatchNormalization()
mpool1 = MaxPooling2D((2,2),padding='same')
conv2 = Conv2D(64,(3,3),padding='same',activation='relu')
bnorm2 = BatchNormalization()
mpool2 = MaxPooling2D((2,2),padding='same')
conv3 = Conv2D(64,(3,3),padding='same',activation='relu')
bnorm3 = BatchNormalization()
mpool3 = MaxPooling2D((2,2),padding='same')
conv4 = Conv2D(64,(3,3),padding='same',activation='relu')
bnorm4 = BatchNormalization()
mpool4 = MaxPooling2D((2,2),padding='same')
fltn = Flatten()

# Function that generarates Deep CNN embedding given the input image x
def convembedding(x):
    x = conv1(x)
    x = bnorm1(x)
    x = mpool1(x)
    x = conv2(x)
    x = bnorm2(x)
    x = mpool2(x)
    x = conv3(x)
    x = bnorm3(x)
    x = mpool3(x)
    x = conv4(x)
    x = bnorm4(x)
    x = mpool4(x)
    x = fltn(x)
    
    return x

# Relational embedding comprising a 4 layer MLP
d1 = Dense(64,activation='relu')
dbnrm1 = BatchNormalization()
d2 = Dense(64,activation='relu')
dbnrm2 = BatchNormalization()
d3 = Dense(64,activation='relu')
dbnrm3 = BatchNormalization()
d4 = Dense(64,activation='relu')
dbnrm4 = BatchNormalization()

def relationalembedding(x):
    x = d1(x)
    x = dbnrm1(x)
    x = d2(x)
    x = dbnrm2(x)
    x = d3(x)
    x = dbnrm3(x)
    x = d4(x)
    x = dbnrm4(x)
    
    return x

numsupportset = samples_per_class*classes_per_set
input1 = Input((numsupportset+1,28,28,1))

# CNN embedding support set and query image
convolutionlayers = []
for lidx in range(numsupportset):
    convolutionlayers.append(convembedding(Lambda(lambda x: x[:,lidx,:,:,:])(input1)))
targetembedding = convembedding(Lambda(lambda x: x[:,-1,:,:,:])(input1))

# Siamese like pairwise interactions
siam = Siamify()
pairwiseinteractions = defaultdict(list)

# Get all pairwise Siamese interactions in the support set and generate of list of interactions
# for each member of support set 
for tt in combinations(range(numsupportset),2):
    aa = siam([convolutionlayers[tt[0]],convolutionlayers[tt[1]]])
    pairwiseinteractions[tt[0]].append(aa)
    pairwiseinteractions[tt[1]].append(aa)

# Get Siamese interactions for query image
targetinteractions = []
for i in range(numsupportset):
    aa = siam([targetembedding,convolutionlayers[i]])
    targetinteractions.append(aa)  
    pairwiseinteractions[i].append(aa) # add this interaction to the set of interaction for this member

# Take 4 layer MLP transform on Max pooling of interactions to serve as Full Context Embedding (FCE)
maxi = Maximum()
modelinputs = []
for i in range(numsupportset):
    modelinputs.append(relationalembedding(maxi(pairwiseinteractions[i])))
modelinputs.append(relationalembedding(maxi(targetinteractions)))

supportlabels = Input((numsupportset,classes_per_set))
modelinputs.append(supportlabels)

knnsimilarity = MatchEuclidean(nway=classes_per_set)(modelinputs)

model = Model(inputs=[input1,supportlabels],outputs=knnsimilarity)
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

model.fit([data.datasets_cache["train"][0],data.datasets_cache["train"][1]],data.datasets_cache["train"][2],
          validation_data=[[data.datasets_cache["val"][0],data.datasets_cache["val"][1]],data.datasets_cache["val"][2]],
          epochs=10,batch_size=32,verbose=1)

train_shape (1200, 20, 28, 28, 1) val_shape (422, 20, 28, 28, 1)
before_normalization mean 0.9189295 max 1.0 min 0.0 std 0.21971507
after_normalization mean -1.9403703e-06 max 0.36898008 min -4.182369 std 0.99999905




Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor

Train on 64000 samples, validate on 20000 samples
Epoch 1/10





64000/64000 [==============================] - 276s - loss: 1.1340 - acc: 0.5514 - val_loss: 0.7229 - val_acc: 0.7244
Epoch 2/10
64000/64000 [==============================] - 263s - loss: 0.6193 - acc: 0.7646 - val_loss: 0.3859 - val_acc: 0.8572
Epoch 3/10
64000/64000 [==============================] - 263s - loss: 0.4294 - acc: 0.8408 - val_loss: 0

KeyboardInterrupt: ignored